# BrainScanAI - Analyse Exploratoire

## Notebook 1: Analyse exploratoire des données IRM et clustering pour labellisation faible

### Objectifs
1. Charger et explorer les données IRM
2. Extraire des features avec ResNet pré-entraîné
3. Réduire la dimensionnalité avec PCA/t-SNE
4. Appliquer des algorithmes de clustering (K-Means, DBSCAN)
5. Générer des labels faibles pour l'apprentissage semi-supervisé

### Prérequis
- Python 3.11+
- PyTorch/Torchvision
- Scikit-learn
- Matplotlib/Seaborn

### Structure du notebook
1. Configuration de l'environnement
2. Chargement des données
3. Préprocessing des images
4. Extraction de features
5. Analyse de dimensionnalité
6. Clustering
7. Visualisation des résultats
8. Génération de labels faibles

## 1. Configuration de l'environnement

In [ ]:
# Import des bibliothèques
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# ML et Deep Learning
import torch
import torchvision
from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset
import sklearn
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.preprocessing import StandardScaler

# Visualisation
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Configuration
DATA_DIR = Path('../data/raw')
PROCESSED_DIR = Path('../data/processed')
MODELS_DIR = Path('../models')
RESULTS_DIR = Path('../results')

# Création des répertoires
for dir_path in [DATA_DIR, PROCESSED_DIR, MODELS_DIR, RESULTS_DIR]:
    dir_path.mkdir(parents=True, exist_ok=True)

print(f"Python version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"Torchvision version: {torchvision.__version__}")
print(f"Scikit-learn version: {sklearn.__version__}")
print(f"GPU disponible: {torch.cuda.is_available()}")

## 2. Chargement des données

**Note**: Ce notebook suppose que les données IRM sont disponibles dans `data/raw/`. 
Pour utiliser des données de test, nous allons générer des données synthétiques ou utiliser un dataset public comme BraTS.

In [ ]:
# Fonction pour charger les données IRM
def load_mri_data(data_dir):
    """
    Charge les données IRM depuis le répertoire spécifié.
    
    Args:
        data_dir (Path): Chemin vers le répertoire des données
        
    Returns:
        images (list): Liste des chemins d'images
        labels (list): Liste des labels (si disponibles)
        metadata (DataFrame): Métadonnées des images
    """
    # À implémenter selon la structure des données
    # Pour l'instant, retourne des données factices
    
    # Exemple avec données synthétiques pour la démo
    print(f"Recherche des données dans: {data_dir}")
    
    # Vérification de l'existence des données
    if not data_dir.exists():
        print(f"Le répertoire {data_dir} n'existe pas. Création...")
        data_dir.mkdir(parents=True, exist_ok=True)
        
        # Création de données synthétiques pour la démo
        print("Création de données synthétiques pour la démonstration...")
        # Cette partie serait remplacée par le chargement réel des données
        
    return [], [], pd.DataFrame()

# Chargement des données
images, labels, metadata = load_mri_data(DATA_DIR)
print(f"Nombre d'images chargées: {len(images)}")
print(f"Nombre de labels: {len(labels) if labels else 'Aucun'}")
print(f"Métadonnées: {metadata.shape if not metadata.empty else 'Aucune'}")

## 3. Préprocessing des images

Prétraitement standard pour ResNet: redimensionnement, normalisation, augmentation.

In [ ]:
# Transformations pour ResNet
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Taille d'entrée ResNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet stats
                         std=[0.229, 0.224, 0.225])
])

# Dataset personnalisé pour les IRM
class MRIDataset(Dataset):
    def __init__(self, image_paths, labels=None, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        # À implémenter: charger l'image IRM
        # Pour l'instant, retourne un tensor factice
        image = torch.randn(3, 224, 224)  # Image factice
        
        if self.transform:
            image = self.transform(image)
            
        if self.labels is not None:
            label = self.labels[idx]
            return image, label
        
        return image

print("Dataset et transformations définis")

## 4. Extraction de features avec ResNet

Utilisation d'un ResNet pré-entraîné sur ImageNet comme extracteur de features.

In [ ]:
# Chargement du modèle ResNet pré-entraîné
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(pretrained=True)
model = model.to(device)
model.eval()  # Mode évaluation

# Modification pour extraire les features (supprimer la dernière couche)
feature_extractor = torch.nn.Sequential(*list(model.children())[:-1])
feature_extractor.eval()

print(f"Modèle chargé sur: {device}")
print(f"Nombre de paramètres: {sum(p.numel() for p in model.parameters()):,}")

## 5. Analyse de dimensionnalité avec PCA et t-SNE

Réduction de dimensionnalité pour visualiser la structure des données.

In [ ]:
# Fonction pour extraire les features
def extract_features(dataset, feature_extractor, device, batch_size=32):
    """
    Extrait les features d'un dataset d'images.
    """
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    features = []
    
    with torch.no_grad():
        for batch in dataloader:
            if isinstance(batch, tuple):
                images = batch[0]
            else:
                images = batch
                
            images = images.to(device)
            batch_features = feature_extractor(images)
            batch_features = batch_features.view(batch_features.size(0), -1)
            features.append(batch_features.cpu().numpy())
    
    return np.vstack(features)

# Exemple d'extraction (à adapter avec des données réelles)
print("Extraction de features... (exemple avec données factices)")
# features = extract_features(dataset, feature_extractor, device)
# print(f"Shape des features: {features.shape}")

## 6. Clustering pour labellisation faible

Application de K-Means et DBSCAN pour générer des labels automatiques.

In [ ]:
# Fonction pour déterminer le nombre optimal de clusters (méthode elbow)
def find_optimal_clusters(features, max_k=10):
    """
    Trouve le nombre optimal de clusters avec la méthode elbow.
    """
    inertias = []
    K = range(1, max_k + 1)
    
    for k in K:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        kmeans.fit(features)
        inertias.append(kmeans.inertia_)
    
    return inertias, K

# Fonction pour évaluer la qualité des clusters
def evaluate_clusters(features, labels):
    """
    Évalue la qualité des clusters avec différentes métriques.
    """
    silhouette = silhouette_score(features, labels)
    davies_bouldin = davies_bouldin_score(features, labels)
    
    return {
        'silhouette_score': silhouette,
        'davies_bouldin_score': davies_bouldin
    }

print("Fonctions de clustering définies")

## 7. Visualisation des résultats

Visualisation des clusters et analyse des résultats.

In [ ]:
# Fonctions de visualisation
def plot_clusters_2d(features_2d, cluster_labels, title="Clusters"):
    """
    Visualise les clusters en 2D.
    """
    plt.figure(figsize=(10, 8))
    scatter = plt.scatter(features_2d[:, 0], features_2d[:, 1], 
                         c=cluster_labels, cmap='viridis', alpha=0.6)
    plt.colorbar(scatter)
    plt.title(title)
    plt.xlabel("Dimension 1")
    plt.ylabel("Dimension 2")
    plt.grid(True, alpha=0.3)
    plt.show()

def plot_elbow_method(inertias, K):
    """
    Trace la méthode elbow pour déterminer le k optimal.
    """
    plt.figure(figsize=(10, 6))
    plt.plot(K, inertias, 'bo-')
    plt.xlabel('Nombre de clusters (k)')
    plt.ylabel('Inertie')
    plt.title('Méthode Elbow pour déterminer k optimal')
    plt.grid(True)
    plt.show()

print("Fonctions de visualisation définies")

## 8. Génération de labels faibles

Création d'un dataset faiblement labellisé pour l'apprentissage semi-supervisé.

In [ ]:
# Fonction pour générer les labels faibles
def generate_weak_labels(features, n_clusters=5):
    """
    Génère des labels faibles via clustering.
    """
    # Standardisation des features
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)
    
    # Réduction de dimensionnalité avec PCA
    pca = PCA(n_components=50)  # Réduction à 50 dimensions
    features_pca = pca.fit_transform(features_scaled)
    
    # Clustering avec K-Means
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(features_pca)
    
    # Évaluation
    metrics = evaluate_clusters(features_pca, cluster_labels)
    
    # Réduction à 2D pour visualisation
    tsne = TSNE(n_components=2, random_state=42)
    features_2d = tsne.fit_transform(features_pca)
    
    return {
        'weak_labels': cluster_labels,
        'features_2d': features_2d,
        'metrics': metrics,
        'pca': pca,
        'kmeans': kmeans,
        'scaler': scaler
    }

print("Fonction de génération de labels faibles définie")

## 9. Pipeline complet

Exécution du pipeline complet d'analyse exploratoire.

In [ ]:
# Pipeline complet
def exploratory_pipeline(data_dir, n_clusters=5):
    """
    Pipeline complet d'analyse exploratoire.
    """
    print("=== Début du pipeline d'analyse exploratoire ===")
    
    # 1. Chargement des données
    print("1. Chargement des données...")
    images, labels, metadata = load_mri_data(data_dir)
    
    # 2. Création du dataset
    print("2. Création du dataset...")
    dataset = MRIDataset(images, labels, transform=transform)
    
    # 3. Extraction de features
    print("3. Extraction de features avec ResNet...")
    features = extract_features(dataset, feature_extractor, device)
    print(f"   Features extraits: {features.shape}")
    
    # 4. Génération de labels faibles
    print("4. Génération de labels faibles via clustering...")
    results = generate_weak_labels(features, n_clusters=n_clusters)
    
    # 5. Visualisation
    print("5. Visualisation des résultats...")
    plot_clusters_2d(results['features_2d'], results['weak_labels'], 
                    title=f"Clusters (k={n_clusters})")
    
    # 6. Métriques
    print("6. Métriques d'évaluation:")
    for metric, value in results['metrics'].items():
        print(f"   {metric}: {value:.4f}")
    
    print("=== Pipeline terminé ===")
    
    return results

# Exécution du pipeline
# results = exploratory_pipeline(DATA_DIR, n_clusters=5)

## 10. Sauvegarde des résultats

Sauvegarde des features et labels faibles pour les étapes suivantes.

In [ ]:
# Fonction pour sauvegarder les résultats
def save_results(results, output_dir):
    """
    Sauvegarde les résultats de l'analyse exploratoire.
    """
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Sauvegarde des labels faibles
    weak_labels_path = output_dir / "weak_labels.npy"
    np.save(weak_labels_path, results['weak_labels'])
    
    # Sauvegarde des features
    features_path = output_dir / "features.npy"
    # np.save(features_path, features)  # À remplacer par les vraies features
    
    # Sauvegarde des métriques
    metrics_path = output_dir / "clustering_metrics.json"
    import json
    with open(metrics_path, 'w') as f:
        json.dump(results['metrics'], f, indent=2)
    
    # Sauvegarde des visualisations
    plot_path = output_dir / "clusters_plot.png"
    plt.savefig(plot_path, dpi=300, bbox_inches='tight')
    
    print(f"Résultats sauvegardés dans: {output_dir}")
    
    return {
        'weak_labels': weak_labels_path,
        'metrics': metrics_path,
        'plot': plot_path
    }

# Sauvegarde exemple
# save_results(results, RESULTS_DIR / "exploratory_analysis")

## Conclusion

Ce notebook fournit le pipeline complet pour:
1. L'analyse exploratoire des données IRM
2. L'extraction de features avec ResNet
3. La réduction de dimensionnalité avec PCA/t-SNE
4. Le clustering pour la génération de labels faibles
5. La visualisation et l'évaluation des résultats

**Prochaines étapes**:
1. Remplacer les données factices par des données IRM réelles
2. Ajuster les hyperparamètres de clustering
3. Tester différents algorithmes de clustering
4. Valider les labels faibles avec un sous-ensemble de labels experts

**Livrable**: Ce notebook constitue le Livrable 1 (Analyse exploratoire et clustering).